In this notebook, for all three datasets and various L infinity attacks, we evaluate the performnace of each component of our detector to study how they perform on their own. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib
import matplotlib.pyplot as plt
import gc
from captum.attr import *
import quantus
from torch.utils.data import DataLoader
import gc
import torchvision.transforms as transforms
import os
import torch.optim as optim
import torchvision.models as models

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda')

In [9]:
def make_noise(x_batch, y_batch, spread):
    new_x_batch = []
    for x in x_batch:
        x = x.data.cpu().numpy()
        stdev = spread * (np.max(x)-np.min(x))
        noise = np.random.normal(0, stdev, x.shape).astype(np.float32)
        x_plus_noise = x + noise
        x_plus_noise = np.clip(x_plus_noise, 0, 1)
        x_plus_noise = torch.from_numpy(x_plus_noise).cpu()
        new_x_batch.append(x_plus_noise)
    new_batch = torch.stack(new_x_batch).to(device)
    return new_batch

In [10]:
def compute_metrics_benign(adv_path, normal_model): 
    
    print("Computing metrics for {} for benign")
    
    npobj = np.load(adv_path)
    adaptive_image = npobj['b_images']
    adaptive_label = npobj['b_labels']
    
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    
    images, labels = torch.from_numpy(adaptive_image), torch.from_numpy(adaptive_label)
    #images, labels = images.to(device), labels.to(device)
    
    end = len(adaptive_label)
    if end > 1000:
        end = 1000
    
    for i in range(0, end, 2):
        
        images_adv, y_pred_adv = images[i:i+2], labels[i:i+2]
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        x_logits = normal_model(images_adv)
        gc.collect()
        torch.cuda.empty_cache()
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.005)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.01)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.05)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        
    df = pd.DataFrame([
            
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3], index = [
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
                    ])
            
    path = "adaptive_Benign.csv"
    df.to_csv(path)

In [11]:
def compute_metrics_adv(adv_path, normal_model): 
    
    print("Computing metrics for {} for adv")
    npobj = np.load(adv_path)
    adaptive_image = npobj['a_images']
    adaptive_label = npobj['a_labels']
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    
    images, labels = torch.from_numpy(adaptive_image), torch.from_numpy(adaptive_label)
    #images, labels = images.to(device), labels.to(device)
    
    end = len(adaptive_label)
    if end > 1000:
        end = 1000
    
    for i in range(0, end, 2):
        
        images_adv, y_pred_adv = images[i:i+2], labels[i:i+2]
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        x_logits = normal_model(images_adv)
        gc.collect()
        torch.cuda.empty_cache()
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.005)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.01)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.05)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        
    df = pd.DataFrame([
            
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3], index = [
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
                    ])
            
    path = "adaptive_Adv.csv"
    df.to_csv(path)

In [25]:
def compute_TPR(adv1, a, b): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1 in adv1: 
        if value1<a or value1>b:
            TP += 1
        else:
            FN+=1
    
    return (TP/(TP+FN))*100

In [26]:
def compute_FPR(ap2a, k, l): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value6 in ap2a:
        if value6<k or value6>l:
            FP +=1

    return (FP/(len(ap2a)))*100

In [27]:
import sklearn
from sklearn.metrics import roc_auc_score

# MNIST

In [5]:
#for natural and adversarial LeNet Model 
class LeNet_normal(torch.nn.Module):
    """Network architecture from: https://github.com/ChawDoe/LeNet5-MNIST-PyTorch."""
    def __init__(self):
        super().__init__()
        self.conv_1 = torch.nn.Conv2d(1, 6, 5)
        self.pool_1 = torch.nn.MaxPool2d(2, 2)
        self.relu_1 = torch.nn.ReLU()
        self.conv_2 = torch.nn.Conv2d(6, 16, 5)
        self.pool_2 = torch.nn.MaxPool2d(2, 2)
        self.relu_2 = torch.nn.ReLU()
        self.fc_1 = torch.nn.Linear(256, 120)
        self.relu_3 = torch.nn.ReLU()
        self.fc_2 = torch.nn.Linear(120, 84)
        self.relu_4 = torch.nn.ReLU()
        self.fc_3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool_1(self.relu_1(self.conv_1(x)))
        x = self.pool_2(self.relu_2(self.conv_2(x)))
        x = x.view(x.shape[0], -1)
        x = self.relu_3(self.fc_1(x))
        x = self.relu_4(self.fc_2(x))
        x = self.fc_3(x)
        return x

In [6]:
def load_mnist_model(path):
    model = LeNet_normal()
    model.to(device)
    model.load_state_dict(torch.load(path))
    model.to('cuda')
    model.train(False)
    return model

In [7]:
modelpath = "/data/virtual environments/adv detection by robustness/adv_detection/Adaptive attacks/Models/MNIST/mnist_model.pth"

In [8]:
normal_model = load_mnist_model(modelpath)
normal_model.to(device)
normal_model.eval()

LeNet_normal(
  (conv_1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu_1): ReLU()
  (conv_2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu_2): ReLU()
  (fc_1): Linear(in_features=256, out_features=120, bias=True)
  (relu_3): ReLU()
  (fc_2): Linear(in_features=120, out_features=84, bias=True)
  (relu_4): ReLU()
  (fc_3): Linear(in_features=84, out_features=10, bias=True)
)

In [21]:
def return_auc_logit(adv_path, model):
    #logitgaussian3
    k=[0.07,0.07,0.07, 0.07,0.07,0.07,0.07,0.07,0.07, 0.07]
    l=[3.3,2.1,1.7,1.5, 1.3, 1.1, 0.7, 0.5, 0.3, 0.1]

    compute_metrics_benign(adv_path, model)
    compute_metrics_adv(adv_path, model)
    df_cifar = pd.read_csv("adaptive_Benign.csv")
    attr_gaussian3 = df_cifar.iloc[0].values.flatten().tolist()[1:]
    logit_gaussian3 = df_cifar.iloc[3].values.flatten().tolist()[1:]
        
    fpr_results =[]
    for t1,t2 in zip(k,l):
        FPR = compute_FPR(logit_gaussian3, t1,t2)
        fpr_results.append(FPR/100)
        
    df_pgd_eps1 = pd.read_csv("adaptive_Adv.csv")
    attr_gaussian3_eps1 = df_pgd_eps1.iloc[0].values.flatten().tolist()[1:]
    logit_gaussian3_eps1 = df_pgd_eps1.iloc[3].values.flatten().tolist()[1:]
    
    tpr_results =[]
    for t1,t2 in zip(k,l):
        TPR = compute_TPR(logit_gaussian3_eps1, t1,t2)
        tpr_results.append(TPR/100)
    return(sklearn.metrics.auc(fpr_results, tpr_results), tpr_results, fpr_results)

In [22]:
def return_auc_attr(adv_path, model):
    
    #attr
    m=[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
    n=[250,150,120, 110, 100, 90, 80, 70, 30,10]

    
    compute_metrics_benign(adv_path, model)
    compute_metrics_adv(adv_path, model)
    df_cifar = pd.read_csv("adaptive_Benign.csv")
    attr_gaussian3 = df_cifar.iloc[0].values.flatten().tolist()[1:]
    logit_gaussian3 = df_cifar.iloc[3].values.flatten().tolist()[1:]
        
    fpr_results =[]
    for t3,t4 in zip(m,n):
        FPR = compute_FPR(attr_gaussian3,t3,t4)
        fpr_results.append(FPR/100)
        
    df_pgd_eps1 = pd.read_csv("adaptive_Adv.csv")
    attr_gaussian3_eps1 = df_pgd_eps1.iloc[0].values.flatten().tolist()[1:]
    logit_gaussian3_eps1 = df_pgd_eps1.iloc[3].values.flatten().tolist()[1:]
    
    tpr_results =[]
    for t3,t4 in zip(m,n):
        TPR = compute_TPR(attr_gaussian3_eps1,t3,t4)
        tpr_results.append(TPR/100)
    return(sklearn.metrics.auc(fpr_results, tpr_results), tpr_results, fpr_results)

# FGSM

In [23]:
adv_path1 = 'adv samples/MNIST/FGSM/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/MNIST/FGSM/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/MNIST/FGSM/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/MNIST/FGSM/0.25098039215686274eps.npz'

In [28]:
auc, tpr, fpr = return_auc_logit(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.975626,
 [0.879,
  0.968,
  0.9789999999999999,
  0.982,
  0.985,
  0.9940000000000001,
  0.9990000000000001,
  1.0,
  1.0,
  1.0],
 [0.012,
  0.05600000000000001,
  0.109,
  0.15,
  0.20800000000000002,
  0.296,
  0.534,
  0.714,
  0.8909999999999999,
  0.996])

In [29]:
auc, tpr, fpr = return_auc_logit(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9852145000000001,
 [0.955,
  0.9890000000000001,
  0.997,
  0.9990000000000001,
  0.9990000000000001,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 [0.011, 0.055, 0.105, 0.159, 0.218, 0.291, 0.542, 0.709, 0.887, 0.998])

In [30]:
auc, tpr, fpr = return_auc_logit(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.986434,
 [0.976,
  0.993,
  0.996,
  0.9990000000000001,
  0.9990000000000001,
  0.9990000000000001,
  1.0,
  1.0,
  1.0,
  1.0],
 [0.01,
  0.067,
  0.12300000000000001,
  0.172,
  0.237,
  0.308,
  0.54,
  0.716,
  0.895,
  0.998])

In [31]:
auc, tpr, fpr = return_auc_logit(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9909795000000001,
 [0.9990000000000001, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.008,
  0.049,
  0.105,
  0.155,
  0.218,
  0.299,
  0.573,
  0.737,
  0.904,
  0.9990000000000001])

In [32]:
auc, tpr, fpr = return_auc_attr(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.8306735000000001,
 [0.355,
  0.764,
  0.8809999999999999,
  0.91,
  0.939,
  0.955,
  0.972,
  0.986,
  1.0,
  1.0],
 [0.009,
  0.045,
  0.077,
  0.096,
  0.12300000000000001,
  0.159,
  0.2,
  0.24600000000000002,
  0.588,
  0.872])

In [33]:
auc, tpr, fpr = return_auc_attr(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.8679795000000003,
 [0.568,
  0.9010000000000001,
  0.946,
  0.958,
  0.97,
  0.983,
  0.988,
  0.9940000000000001,
  0.9990000000000001,
  1.0],
 [0.02,
  0.045,
  0.072,
  0.087,
  0.10100000000000002,
  0.128,
  0.168,
  0.215,
  0.586,
  0.9010000000000001])

In [34]:
auc, tpr, fpr = return_auc_attr(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.862923,
 [0.757,
  0.953,
  0.976,
  0.983,
  0.9890000000000001,
  0.99,
  0.993,
  0.997,
  1.0,
  1.0],
 [0.012,
  0.039,
  0.076,
  0.09300000000000001,
  0.10800000000000001,
  0.129,
  0.16699999999999998,
  0.214,
  0.555,
  0.882])

In [35]:
auc, tpr, fpr = return_auc_attr(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.8642335000000001,
 [0.892,
  0.99,
  0.997,
  0.998,
  0.9990000000000001,
  0.9990000000000001,
  0.9990000000000001,
  0.9990000000000001,
  1.0,
  1.0],
 [0.013000000000000001,
  0.05,
  0.086,
  0.096,
  0.11899999999999998,
  0.155,
  0.199,
  0.24600000000000002,
  0.572,
  0.88])

# PGD

In [36]:
adv_path1 = 'adv samples/MNIST/PGD/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/MNIST/PGD/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/MNIST/PGD/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/MNIST/PGD/0.25098039215686274eps.npz'

In [37]:
auc, tpr, fpr = return_auc_logit(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.988,
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.009,
  0.054000000000000006,
  0.09700000000000002,
  0.138,
  0.206,
  0.282,
  0.544,
  0.706,
  0.89,
  0.997])

In [38]:
auc, tpr, fpr = return_auc_logit(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9870000000000002,
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.012,
  0.064,
  0.118,
  0.16699999999999998,
  0.22,
  0.302,
  0.565,
  0.7340000000000001,
  0.904,
  0.9990000000000001])

In [39]:
auc, tpr, fpr = return_auc_logit(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.991,
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.009,
  0.054000000000000006,
  0.107,
  0.15,
  0.223,
  0.297,
  0.546,
  0.728,
  0.902,
  1.0])

In [40]:
auc, tpr, fpr = return_auc_logit(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9820000000000001,
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.012,
  0.05800000000000001,
  0.109,
  0.151,
  0.215,
  0.293,
  0.515,
  0.723,
  0.899,
  0.9940000000000001])

In [41]:
auc, tpr, fpr = return_auc_attr(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.857496,
 [0.61,
  0.915,
  0.961,
  0.975,
  0.986,
  0.9890000000000001,
  0.993,
  0.996,
  1.0,
  1.0],
 [0.013000000000000001,
  0.043,
  0.081,
  0.106,
  0.136,
  0.16400000000000003,
  0.204,
  0.256,
  0.577,
  0.883])

In [42]:
auc, tpr, fpr = return_auc_attr(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.852052,
 [0.8030000000000002,
  0.968,
  0.991,
  0.992,
  0.9940000000000001,
  0.9990000000000001,
  1.0,
  1.0,
  1.0,
  1.0],
 [0.015,
  0.049,
  0.08200000000000002,
  0.09300000000000001,
  0.118,
  0.145,
  0.176,
  0.228,
  0.554,
  0.872])

In [43]:
auc, tpr, fpr = return_auc_attr(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.8787875,
 [0.934,
  0.988,
  0.995,
  0.9990000000000001,
  0.9990000000000001,
  0.9990000000000001,
  0.9990000000000001,
  0.9990000000000001,
  1.0,
  1.0],
 [0.012,
  0.052000000000000005,
  0.086,
  0.103,
  0.11899999999999998,
  0.14,
  0.179,
  0.233,
  0.598,
  0.893])

In [44]:
auc, tpr, fpr = return_auc_attr(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.8825665,
 [0.972, 0.9990000000000001, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.011, 0.04, 0.066, 0.083, 0.103, 0.138, 0.18, 0.23, 0.58, 0.894])

# BIM

In [45]:
adv_path1 = 'adv samples/MNIST/BIM/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/MNIST/BIM/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/MNIST/BIM/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/MNIST/BIM/0.25098039215686274eps.npz'

In [46]:
auc, tpr, fpr = return_auc_logit(adv_path1, normal_model)
auc, tpr, fpr 

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.588911,
 [0.021,
  0.1,
  0.16400000000000003,
  0.215,
  0.305,
  0.3990000000000001,
  0.687,
  0.816,
  0.949,
  0.9990000000000001],
 [0.008, 0.042, 0.094, 0.125, 0.191, 0.281, 0.557, 0.732, 0.898, 0.998])

In [47]:
auc, tpr, fpr = return_auc_logit(adv_path2, normal_model)
auc, tpr, fpr 

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.5411635,
 [0.018,
  0.071,
  0.129,
  0.192,
  0.251,
  0.368,
  0.616,
  0.7829999999999999,
  0.929,
  0.9990000000000001],
 [0.006,
  0.047,
  0.10100000000000002,
  0.157,
  0.21899999999999997,
  0.291,
  0.56,
  0.74,
  0.907,
  0.998])

In [48]:
auc, tpr, fpr = return_auc_logit(adv_path3, normal_model)
auc, tpr, fpr 

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.5260334999999999,
 [0.036,
  0.088,
  0.141,
  0.184,
  0.262,
  0.345,
  0.574,
  0.7609999999999999,
  0.911,
  0.992],
 [0.012,
  0.061,
  0.114,
  0.163,
  0.218,
  0.282,
  0.545,
  0.7289999999999999,
  0.9060000000000001,
  0.996])

In [49]:
auc, tpr, fpr = return_auc_logit(adv_path4, normal_model)
auc, tpr, fpr 

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.5577964999999999,
 [0.036,
  0.094,
  0.174,
  0.231,
  0.289,
  0.37799999999999995,
  0.62,
  0.76,
  0.9129999999999999,
  0.996],
 [0.006,
  0.05,
  0.09,
  0.128,
  0.20200000000000004,
  0.284,
  0.556,
  0.723,
  0.893,
  1.0])

In [50]:
auc, tpr, fpr = return_auc_attr(adv_path1, normal_model)
auc, tpr, fpr 

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.481561,
 [0.026000000000000002,
  0.08900000000000001,
  0.147,
  0.174,
  0.214,
  0.241,
  0.282,
  0.333,
  0.684,
  0.935],
 [0.013000000000000001,
  0.045,
  0.087,
  0.10400000000000001,
  0.129,
  0.15,
  0.184,
  0.21600000000000003,
  0.56,
  0.892])

In [51]:
auc, tpr, fpr = return_auc_attr(adv_path2, normal_model)
auc, tpr, fpr 

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.42670450000000004,
 [0.01,
  0.057,
  0.111,
  0.133,
  0.158,
  0.18899999999999997,
  0.23799999999999996,
  0.28,
  0.652,
  0.9280000000000002],
 [0.013000000000000001,
  0.048,
  0.091,
  0.105,
  0.12300000000000001,
  0.157,
  0.201,
  0.245,
  0.599,
  0.885])

In [52]:
auc, tpr, fpr = return_auc_attr(adv_path3, normal_model)
auc, tpr, fpr 

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.41449100000000005,
 [0.02,
  0.077,
  0.111,
  0.133,
  0.16,
  0.18899999999999997,
  0.225,
  0.273,
  0.605,
  0.919],
 [0.015,
  0.039,
  0.078,
  0.09700000000000002,
  0.11899999999999998,
  0.155,
  0.2,
  0.23799999999999996,
  0.579,
  0.88])

In [53]:
auc, tpr, fpr = return_auc_attr(adv_path4, normal_model)
auc, tpr, fpr 

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.3855050000000001,
 [0.019,
  0.04100000000000001,
  0.071,
  0.096,
  0.12,
  0.143,
  0.179,
  0.21899999999999997,
  0.568,
  0.867],
 [0.017,
  0.045,
  0.081,
  0.105,
  0.12,
  0.149,
  0.18099999999999997,
  0.223,
  0.597,
  0.8960000000000001])

# CW

In [54]:
adv_path1 = 'adv samples/MNIST/CW/cwMNIST.npz'

auc, tpr, fpr = return_auc_logit(adv_path1, normal_model)
auc, tpr, fpr 

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.392212,
 [0.008,
  0.024,
  0.036,
  0.072,
  0.096,
  0.176,
  0.376,
  0.584,
  0.8320000000000001,
  0.9840000000000001],
 [0.007000000000000001,
  0.053,
  0.122,
  0.163,
  0.214,
  0.29,
  0.555,
  0.701,
  0.889,
  0.995])

In [55]:
auc, tpr, fpr = return_auc_attr(adv_path1, normal_model)
auc, tpr, fpr 

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.37232,
 [0.016,
  0.05600000000000001,
  0.076,
  0.096,
  0.10400000000000001,
  0.144,
  0.176,
  0.22,
  0.524,
  0.868],
 [0.012,
  0.04100000000000001,
  0.078,
  0.096,
  0.122,
  0.138,
  0.176,
  0.221,
  0.557,
  0.877])

# CIFAR-10

In [56]:
#from rev2.cifar10.model_utils import resnet50, CIFAR10_RESNET50_CKPT_PATH

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, out_keys=None):
        out = {}
        x = self.conv1(x)
        out["c1"] = x
        x = self.bn1(x)
        out["bn1"] = x
        x = F.relu(x)
        out["r1"] = x

        x = self.layer1(x)
        out["l1"] = x
        x = self.layer2(x)
        out["l2"] = x
        x = self.layer3(x)
        out["l3"] = x
        x = self.layer4(x)
        out["l4"] = x

        x = F.avg_pool2d(x, 4)
        out["gvp"] = x
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        out["fc"] = x

        if out_keys is None:
            return x
        res = {}
        for key in out_keys:
            res[key] = out[key]
        return res


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])


def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])


def resnet50():
    return ResNet(Bottleneck, [3,4,6,3])


def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])


def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])


def test():
    net = ResNet18()
    y = net(torch.randn(1,3,32,32))
    print(y.size())

In [57]:
def load_cifar_model(path):
    model = resnet50()
    ckpt_dict = torch.load(path, lambda storage, loc: storage)
    model.load_state_dict(ckpt_dict)
    model.to('cuda')
    model.train(False)
    return model

In [58]:
modelpath = "/data/virtual environments/adv detection by robustness/adv_detection/Adaptive attacks/Models/CIFAR10/resnet50/cifar.ckpt"

In [59]:
normal_model = load_cifar_model(modelpath)
normal_model.to(device)
normal_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Bottleneck(
      (

In [60]:
def compute_metrics_benign(adv_path, normal_model): 
    
    print("Computing metrics for {} for benign")
    
    npobj = np.load(adv_path)
    adaptive_image = npobj['b_images']
    adaptive_label = npobj['b_labels']
    
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    
    images, labels = torch.from_numpy(adaptive_image), torch.from_numpy(adaptive_label)
    #images, labels = images.to(device), labels.to(device)
    
    end = len(adaptive_label)
    if end > 1000:
        end = 1000
    
    for i in range(0, end, 2):
        
        images_adv, y_pred_adv = images[i:i+2], labels[i:i+2]
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        x_logits = normal_model(images_adv)
        gc.collect()
        torch.cuda.empty_cache()
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.05)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.10)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        
    df = pd.DataFrame([
            
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3], index = [
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
                    ])
            
    path = "adaptive_Benign.csv"
    df.to_csv(path)

In [61]:
def compute_metrics_adv(adv_path, normal_model): 
    
    print("Computing metrics for {} for adv")
    npobj = np.load(adv_path)
    adaptive_image = npobj['a_images']
    adaptive_label = npobj['a_labels']
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    
    images, labels = torch.from_numpy(adaptive_image), torch.from_numpy(adaptive_label)
    #images, labels = images.to(device), labels.to(device)
    
    end = len(adaptive_label)
    if end > 1000:
        end = 1000
    
    for i in range(0, end, 2):
        
        images_adv, y_pred_adv = images[i:i+2], labels[i:i+2]
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        x_logits = normal_model(images_adv)
        gc.collect()
        torch.cuda.empty_cache()
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.05)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.10)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        
    df = pd.DataFrame([
            
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3], index = [
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
                    ])
            
    path = "adaptive_Adv.csv"
    df.to_csv(path)

In [62]:
def compute_TPR(adv1, a, b): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1 in adv1: 
        if value1<a or value1>b:
            TP += 1
        else:
            FN+=1
    
    return (TP/(TP+FN))*100


def compute_FPR(ap2a, k, l): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value6 in ap2a:
        if value6<k or value6>l:
            FP +=1

    return (FP/(len(ap2a)))*100


In [63]:
def return_auc_logit(adv_path, model):
    #logitgaussian3
    k=[44,64,76, 86, 96, 106, 126, 150, 200, 400]
    l=[405,405,405,405,405,405,405,405,405,405, 405]

    compute_metrics_benign(adv_path, model)
    compute_metrics_adv(adv_path, model)
    df_cifar = pd.read_csv("adaptive_Benign.csv")
    attr_gaussian3 = df_cifar.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3 = df_cifar.iloc[5].values.flatten().tolist()[1:]
        
    fpr_results =[]
    for t1,t2 in zip(k,l):
        FPR = compute_FPR(logit_gaussian3, t1,t2)
        fpr_results.append(FPR/100)
        
    df_pgd_eps1 = pd.read_csv("adaptive_Adv.csv")
    attr_gaussian3_eps1 = df_pgd_eps1.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3_eps1 = df_pgd_eps1.iloc[5].values.flatten().tolist()[1:]
    
    tpr_results =[]
    for t1,t2 in zip(k,l):
        TPR = compute_TPR(logit_gaussian3_eps1, t1,t2)
        tpr_results.append(TPR/100)
    return(sklearn.metrics.auc(fpr_results, tpr_results), tpr_results, fpr_results)


def return_auc_attr(adv_path, model):
    
    #attrgaussian3
    m=[2200,2800,3100, 3500, 4000, 4500, 5000, 5500, 6000, 8000]
    n=[8800,8800,8800, 8800,8800,8800, 8800,8800,8800, 8800]

    
    compute_metrics_benign(adv_path, model)
    compute_metrics_adv(adv_path, model)
    df_cifar = pd.read_csv("adaptive_Benign.csv")
    attr_gaussian3 = df_cifar.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3 = df_cifar.iloc[5].values.flatten().tolist()[1:]
        
    fpr_results =[]
    for t3,t4 in zip(m,n):
        FPR = compute_FPR(attr_gaussian3,t3,t4)
        fpr_results.append(FPR/100)
        
    df_pgd_eps1 = pd.read_csv("adaptive_Adv.csv")
    attr_gaussian3_eps1 = df_pgd_eps1.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3_eps1 = df_pgd_eps1.iloc[5].values.flatten().tolist()[1:]
    
    tpr_results =[]
    for t3,t4 in zip(m,n):
        TPR = compute_TPR(attr_gaussian3_eps1,t3,t4)
        tpr_results.append(TPR/100)
    return(sklearn.metrics.auc(fpr_results, tpr_results), tpr_results, fpr_results)

# FGSM

In [64]:
# FGSM
adv_path1 = 'adv samples/CIFAR/FGSM/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/CIFAR/FGSM/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/CIFAR/FGSM/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/CIFAR/FGSM/0.25098039215686274eps.npz'

In [65]:
auc, tpr, fpr = return_auc_logit(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.7700524999999999,
 [0.032,
  0.199,
  0.37200000000000005,
  0.524,
  0.655,
  0.785,
  0.9079999999999999,
  0.9739999999999999,
  0.9990000000000001,
  1.0],
 [0.016,
  0.05800000000000001,
  0.094,
  0.155,
  0.236,
  0.349,
  0.596,
  0.8030000000000002,
  0.9789999999999999,
  1.0])

In [66]:
auc, tpr, fpr = return_auc_logit(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.961981,
 [0.512, 0.865, 0.949, 0.976, 0.988, 0.997, 1.0, 1.0, 1.0, 1.0],
 [0.014000000000000002,
  0.06,
  0.107,
  0.17800000000000002,
  0.262,
  0.37,
  0.593,
  0.797,
  0.975,
  1.0])

In [67]:
auc, tpr, fpr = return_auc_logit(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9877440000000001,
 [0.99, 0.9990000000000001, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.012,
  0.053,
  0.114,
  0.166,
  0.228,
  0.348,
  0.56,
  0.784,
  0.9840000000000001,
  1.0])

In [68]:
auc, tpr, fpr = return_auc_logit(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.984625,
 [0.991, 0.998, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.015,
  0.075,
  0.12,
  0.176,
  0.265,
  0.3509999999999999,
  0.539,
  0.78,
  0.973,
  1.0])

In [69]:
auc, tpr, fpr = return_auc_attr(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.8690665,
 [0.142,
  0.508,
  0.709,
  0.8690000000000001,
  0.96,
  0.988,
  0.997,
  0.998,
  0.9990000000000001,
  1.0],
 [0.015, 0.067, 0.131, 0.234, 0.405, 0.56, 0.688, 0.79, 0.871, 0.986])

In [70]:
auc, tpr, fpr = return_auc_attr(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.973076,
 [0.9060000000000001, 0.998, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.019,
  0.05800000000000001,
  0.11,
  0.212,
  0.389,
  0.54,
  0.7019999999999998,
  0.8080000000000002,
  0.89,
  0.9940000000000001])

In [71]:
auc, tpr, fpr = return_auc_attr(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.981,
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.011, 0.062, 0.109, 0.21, 0.373, 0.535, 0.684, 0.794, 0.883, 0.992])

In [72]:
auc, tpr, fpr = return_auc_attr(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.98,
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.012,
  0.05600000000000001,
  0.102,
  0.21,
  0.374,
  0.54,
  0.681,
  0.785,
  0.866,
  0.992])

# PGD

In [73]:
adv_path1 = 'adv samples/CIFAR/PGD/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/CIFAR/PGD/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/CIFAR/PGD/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/CIFAR/PGD/0.25098039215686274eps.npz'

In [74]:
auc, tpr, fpr = return_auc_logit(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.047874000000000014,
 [0.024,
  0.024,
  0.024,
  0.024,
  0.024,
  0.024,
  0.026000000000000002,
  0.027000000000000003,
  0.10400000000000001,
  0.998],
 [0.019, 0.065, 0.111, 0.166, 0.247, 0.337, 0.548, 0.784, 0.97, 1.0])

In [75]:
auc, tpr, fpr = return_auc_logit(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.10559300000000008,
 [0.09300000000000001,
  0.09300000000000001,
  0.09300000000000001,
  0.09300000000000001,
  0.09300000000000001,
  0.09300000000000001,
  0.09300000000000001,
  0.09300000000000001,
  0.12,
  0.992],
 [0.019,
  0.065,
  0.127,
  0.177,
  0.256,
  0.348,
  0.586,
  0.802,
  0.9739999999999999,
  1.0])

In [76]:
auc, tpr, fpr = return_auc_logit(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.12107900000000002,
 [0.107,
  0.107,
  0.107,
  0.107,
  0.107,
  0.107,
  0.107,
  0.107,
  0.11600000000000002,
  0.997],
 [0.01,
  0.055,
  0.10400000000000001,
  0.169,
  0.266,
  0.369,
  0.573,
  0.7980000000000002,
  0.968,
  1.0])

In [77]:
auc, tpr, fpr = return_auc_logit(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.03742400000000007,
 [0.025,
  0.025,
  0.025,
  0.025,
  0.025,
  0.025,
  0.025,
  0.025,
  0.027000000000000003,
  0.9990000000000001],
 [0.017,
  0.069,
  0.125,
  0.18,
  0.264,
  0.354,
  0.579,
  0.787,
  0.973,
  0.9990000000000001])

In [78]:
auc, tpr, fpr = return_auc_attr(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.8796484999999999,
 [0.17, 0.555, 0.715, 0.857, 0.94, 0.977, 0.995, 0.9990000000000001, 1.0, 1.0],
 [0.009,
  0.055,
  0.099,
  0.21600000000000003,
  0.36,
  0.524,
  0.677,
  0.786,
  0.861,
  0.983])

In [79]:
auc, tpr, fpr = return_auc_attr(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9370905,
 [0.435, 0.852, 0.9280000000000002, 0.977, 0.996, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.017,
  0.065,
  0.12300000000000001,
  0.252,
  0.396,
  0.569,
  0.699,
  0.809,
  0.875,
  0.986])

In [80]:
auc, tpr, fpr = return_auc_attr(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9721965,
 [0.936, 0.998, 0.9990000000000001, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.018,
  0.069,
  0.117,
  0.214,
  0.355,
  0.544,
  0.6780000000000002,
  0.795,
  0.865,
  0.992])

In [81]:
auc, tpr, fpr = return_auc_attr(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.976,
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.011, 0.055, 0.107, 0.209, 0.39, 0.557, 0.689, 0.806, 0.883, 0.987])

# BIM

In [82]:
adv_path1 = 'adv samples/CIFAR/BIM/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/CIFAR/BIM/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/CIFAR/BIM/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/CIFAR/BIM/0.25098039215686274eps.npz'

In [83]:
auc, tpr, fpr = return_auc_logit(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.08458400000000005,
 [0.015, 0.015, 0.015, 0.015, 0.016, 0.018, 0.025, 0.067, 0.418, 1.0],
 [0.02,
  0.069,
  0.11,
  0.17800000000000002,
  0.255,
  0.349,
  0.564,
  0.795,
  0.9689999999999999,
  1.0])

In [84]:
auc, tpr, fpr = return_auc_logit(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.05157700000000001,
 [0.025,
  0.025,
  0.025,
  0.025,
  0.025,
  0.025,
  0.026000000000000002,
  0.028000000000000004,
  0.157,
  1.0],
 [0.014000000000000002,
  0.071,
  0.111,
  0.169,
  0.24600000000000002,
  0.35,
  0.579,
  0.789,
  0.975,
  1.0])

In [85]:
auc, tpr, fpr = return_auc_logit(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.06093300000000001,
 [0.047, 0.047, 0.047, 0.047, 0.047, 0.047, 0.048, 0.051, 0.099, 0.996],
 [0.014000000000000002,
  0.069,
  0.122,
  0.16699999999999998,
  0.231,
  0.333,
  0.559,
  0.809,
  0.982,
  1.0])

In [86]:
auc, tpr, fpr = return_auc_logit(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.07290450000000007,
 [0.045, 0.045, 0.045, 0.045, 0.045, 0.045, 0.046, 0.053, 0.155, 0.996],
 [0.012,
  0.051,
  0.10100000000000002,
  0.169,
  0.258,
  0.355,
  0.553,
  0.785,
  0.9689999999999999,
  1.0])

In [87]:
auc, tpr, fpr = return_auc_attr(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.8334075000000001,
 [0.114, 0.439, 0.601, 0.759, 0.88, 0.949, 0.977, 0.992, 0.995, 1.0],
 [0.02,
  0.059,
  0.11200000000000002,
  0.215,
  0.381,
  0.534,
  0.68,
  0.792,
  0.866,
  0.991])

In [88]:
auc, tpr, fpr = return_auc_attr(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.887964,
 [0.19400000000000003,
  0.579,
  0.743,
  0.871,
  0.945,
  0.981,
  0.992,
  0.997,
  0.9990000000000001,
  1.0],
 [0.011,
  0.062,
  0.111,
  0.203,
  0.36700000000000005,
  0.546,
  0.687,
  0.786,
  0.866,
  0.9890000000000001])

In [89]:
auc, tpr, fpr = return_auc_attr(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9416415,
 [0.45399999999999996, 0.871, 0.948, 0.98, 0.996, 0.998, 1.0, 1.0, 1.0, 1.0],
 [0.017,
  0.076,
  0.124,
  0.217,
  0.392,
  0.557,
  0.716,
  0.825,
  0.887,
  0.9890000000000001])

In [90]:
auc, tpr, fpr = return_auc_attr(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.974656,
 [0.9010000000000001, 0.982, 0.993, 0.998, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.01,
  0.045,
  0.10100000000000002,
  0.196,
  0.365,
  0.513,
  0.6780000000000002,
  0.777,
  0.851,
  0.988])

# CW

In [91]:
adv_path1 = 'adv samples/CIFAR/CW/cwCIFAR.npz'

In [92]:
auc, tpr, fpr = return_auc_logit(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9486114999999999,
 [0.424, 0.82, 0.945, 0.9789999999999999, 0.991, 0.997, 1.0, 1.0, 1.0, 1.0],
 [0.022,
  0.072,
  0.125,
  0.184,
  0.256,
  0.357,
  0.579,
  0.79,
  0.9789999999999999,
  1.0])

In [93]:
auc, tpr, fpr = return_auc_attr(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9714775,
 [0.931, 0.985, 0.992, 0.996, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.013000000000000001,
  0.063,
  0.114,
  0.199,
  0.36199999999999993,
  0.542,
  0.685,
  0.814,
  0.8759999999999999,
  0.988])

# ImageNet: ResNet

In [94]:
def load_imagenet_model():
    model = models.resnet50(pretrained=True).to(device)
    model.to('cuda')
    model.train(False)
    return model

In [95]:
normal_model = load_imagenet_model()
normal_model.to(device)
normal_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [96]:
# Define uniform noise function
def add_uniform_noise(image):
    # Generate uniform noise with mean 0 and standard deviation 25
    noise = np.random.uniform(low=-0.5, high=0.5, size=image.shape).astype(np.float32)
    noisy_image = np.clip(image + noise, 0, 1).astype(np.uint8)
    return noisy_image

def uniform_noise(x_batch, y_batch): 
    # Convert batch of images to numpy array
    images = x_batch.detach().cpu().numpy().transpose(0, 2, 3, 1) * 1.0
    # Add Poisson noise to each image in the batch
    noisy_images = [add_uniform_noise(image) for image in images]
    # Convert noisy images back to Tensor format
    noisy_inputs = torch.from_numpy(np.array(noisy_images).transpose(0, 3, 1, 2) / 1.0).float()
    return noisy_inputs.to(device)


In [97]:
def compute_metrics_benign(adv_path, normal_model): 
    
    print("Computing metrics for {} for benign")
    
    npobj = np.load(adv_path)
    adaptive_image = npobj['b_images']
    adaptive_label = npobj['b_labels']
    
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    images, labels = torch.from_numpy(adaptive_image), torch.from_numpy(adaptive_label)
    #images, labels = images.to(device), labels.to(device)
    
    end = len(adaptive_label)
    if end > 1000:
        end = 1000
    
    for i in range(0, end, 2):
        
        images_adv, y_pred_adv = images[i:i+2], labels[i:i+2]
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        x_logits = normal_model(images_adv)
        gc.collect()
        torch.cuda.empty_cache()
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.25)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.35)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1) 
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        
    df = pd.DataFrame([
            
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
        attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
        logit_uniform
    ], index = [
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution",
        "uniform attr",
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
        "uniform logit"
                    ])
            
    path = "adaptive_Benign.csv"
    df.to_csv(path)

In [98]:
def compute_metrics_adv(adv_path, normal_model): 
    
    print("Computing metrics for {} for adv")
    npobj = np.load(adv_path)
    adaptive_image = npobj['a_images']
    adaptive_label = npobj['a_labels']
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    images, labels = torch.from_numpy(adaptive_image), torch.from_numpy(adaptive_label)
    #images, labels = images.to(device), labels.to(device)
    
    end = len(adaptive_label)
    if end > 1000:
        end = 1000
    
    for i in range(0, end, 2):
        
        images_adv, y_pred_adv = images[i:i+2], labels[i:i+2]
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        x_logits = normal_model(images_adv)
        gc.collect()
        torch.cuda.empty_cache()
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.25)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.35)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1) 
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        
    df = pd.DataFrame([
            
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
        attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
    logit_uniform], index = [
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution",
        "uniform attr",
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
        "uniform logit"
                    ])
            
    path = "adaptive_Adv.csv"
    df.to_csv(path)

In [99]:
def return_auc_attr(adv_path, model):
    #logitgaussian3
#     k=[1810,2510,2810,3000, 3200,3500, 6000]
#     l=[6700,6700,6700,6700,6700,6700, 6700]
    
   
    
    m=[1900,2200,2400, 2600, 2800, 3000, 3400, 5000,7000 ]
    n=[10894,10894,10894, 10894,10894,10894, 10894, 10894, 10894]

    
    #attrgaussian3
#     m=[1400,1750,1900, 2300, 3200, 3500, 6000]
#     n=[6600,6600,6600,6600,6600,6600,6600]

    
    compute_metrics_benign(adv_path, model)
    compute_metrics_adv(adv_path, model)
    df_cifar = pd.read_csv("adaptive_Benign.csv")
    attr_gaussian3 = df_cifar.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3 = df_cifar.iloc[6].values.flatten().tolist()[1:]
        
    fpr_results =[]
    for t3,t4 in zip(m,n):
        FPR = compute_FPR(attr_gaussian3,t3,t4)
        fpr_results.append(FPR/100)
        
    df_pgd_eps1 = pd.read_csv("adaptive_Adv.csv")
    attr_gaussian3_eps1 = df_pgd_eps1.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3_eps1 = df_pgd_eps1.iloc[6].values.flatten().tolist()[1:]
    
    tpr_results =[]
    for t3,t4 in zip(m,n):
        TPR = compute_TPR( attr_gaussian3_eps1,t3,t4)
        tpr_results.append(TPR/100)
    return(sklearn.metrics.auc(fpr_results, tpr_results), tpr_results, fpr_results)

In [100]:
def return_auc_logit(adv_path, model):
    #logitgaussian3
#     k=[1810,2510,2810,3000, 3200,3500, 6000]
#     l=[6700,6700,6700,6700,6700,6700, 6700]
    
    k=[1300,1490,1580, 1600, 1700, 1800, 1900, 2100]
    l=[2970,2970,2970, 2970, 2970, 2970, 2970, 2970]
    
    
    #attrgaussian3
#     m=[1400,1750,1900, 2300, 3200, 3500, 6000]
#     n=[6600,6600,6600,6600,6600,6600,6600]

    
    compute_metrics_benign(adv_path, model)
    compute_metrics_adv(adv_path, model)
    df_cifar = pd.read_csv("adaptive_Benign.csv")
    attr_gaussian3 = df_cifar.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3 = df_cifar.iloc[6].values.flatten().tolist()[1:]
        
    fpr_results =[]
    for t1,t2 in zip(k,l):
        FPR = compute_FPR(logit_gaussian3, t1,t2)
        fpr_results.append(FPR/100)
        
    df_pgd_eps1 = pd.read_csv("adaptive_Adv.csv")
    attr_gaussian3_eps1 = df_pgd_eps1.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3_eps1 = df_pgd_eps1.iloc[6].values.flatten().tolist()[1:]
    
    tpr_results =[]
    for t1,t2 in zip(k,l):
        TPR = compute_TPR(logit_gaussian3_eps1, t1,t2)
        tpr_results.append(TPR/100)
    return(sklearn.metrics.auc(fpr_results, tpr_results), tpr_results, fpr_results)

# FGSM

In [101]:

adv_path1 = 'adv samples/IMAGENET/ResNet50/FGSM/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/IMAGENET/ResNet50/FGSM/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/IMAGENET/ResNet50/FGSM/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/IMAGENET/ResNet50/FGSM/0.25098039215686274eps.npz'

In [102]:
auc, tpr, fpr = return_auc_logit(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.4847,
 [0.048, 0.193, 0.307, 0.341, 0.478, 0.6510000000000001, 0.778, 0.926],
 [0.019, 0.078, 0.14, 0.166, 0.262, 0.434, 0.599, 0.833])

In [103]:
auc, tpr, fpr = return_auc_logit(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.5535304999999999,
 [0.075, 0.275, 0.42699999999999994, 0.451, 0.617, 0.748, 0.845, 0.958],
 [0.024, 0.08200000000000002, 0.153, 0.169, 0.276, 0.425, 0.587, 0.835])

In [104]:
auc, tpr, fpr = return_auc_logit(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.622082,
 [0.137,
  0.385,
  0.539,
  0.585,
  0.7390000000000001,
  0.8489999999999999,
  0.912,
  0.985],
 [0.019,
  0.09,
  0.16,
  0.179,
  0.289,
  0.42699999999999994,
  0.585,
  0.8370000000000001])

In [105]:
auc, tpr, fpr = return_auc_logit(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.7103645000000001,
 [0.303,
  0.612,
  0.758,
  0.775,
  0.8740000000000001,
  0.9280000000000002,
  0.9739999999999999,
  0.998],
 [0.021, 0.079, 0.148, 0.17, 0.301, 0.444, 0.605, 0.838])

In [106]:
auc, tpr, fpr = return_auc_attr(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.604942,
 [0.059,
  0.12,
  0.18899999999999997,
  0.254,
  0.34,
  0.44299999999999995,
  0.621,
  0.945,
  0.998],
 [0.029000000000000005, 0.068, 0.1, 0.142, 0.206, 0.266, 0.424, 0.82, 0.975])

In [107]:
auc, tpr, fpr = return_auc_attr(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.7241454999999999,
 [0.06,
  0.17299999999999996,
  0.31,
  0.425,
  0.534,
  0.644,
  0.814,
  0.9940000000000001,
  1.0],
 [0.017,
  0.049,
  0.09300000000000001,
  0.142,
  0.196,
  0.251,
  0.41600000000000004,
  0.826,
  0.9689999999999999])

In [108]:
auc, tpr, fpr = return_auc_attr(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.865366,
 [0.187, 0.425, 0.599, 0.736, 0.831, 0.91, 0.9739999999999999, 1.0, 1.0],
 [0.019, 0.055, 0.098, 0.143, 0.199, 0.266, 0.41, 0.8370000000000001, 0.98])

In [109]:
auc, tpr, fpr = return_auc_attr(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9296605,
 [0.648,
  0.861,
  0.93,
  0.9689999999999999,
  0.988,
  0.993,
  0.9990000000000001,
  1.0,
  1.0],
 [0.02,
  0.05800000000000001,
  0.094,
  0.141,
  0.20200000000000004,
  0.279,
  0.43799999999999994,
  0.83,
  0.968])

# PGD

In [110]:
adv_path1 = 'adv samples/IMAGENET/ResNet50/PGD/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/IMAGENET/ResNet50/PGD/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/IMAGENET/ResNet50/PGD/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/IMAGENET/ResNet50/PGD/0.25098039215686274eps.npz'

In [111]:
auc, tpr, fpr = return_auc_logit(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.781898,
 [0.986, 0.986, 0.986, 0.986, 0.986, 0.986, 0.986, 0.986],
 [0.02, 0.073, 0.129, 0.148, 0.251, 0.4040000000000001, 0.536, 0.813])

In [112]:
auc, tpr, fpr = return_auc_logit(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.8089999999999999,
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.013000000000000001,
  0.073,
  0.158,
  0.175,
  0.287,
  0.436,
  0.585,
  0.8219999999999998])

In [113]:
auc, tpr, fpr = return_auc_logit(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.837,
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.018, 0.079, 0.161, 0.174, 0.299, 0.45300000000000007, 0.631, 0.855])

In [114]:
auc, tpr, fpr = return_auc_logit(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.8240000000000001,
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.014000000000000002,
  0.072,
  0.156,
  0.177,
  0.3,
  0.44800000000000006,
  0.586,
  0.838])

In [115]:
auc, tpr, fpr = return_auc_attr(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.23297599999999996,
 [0.121, 0.12300000000000001, 0.124, 0.128, 0.135, 0.143, 0.157, 0.344, 0.642],
 [0.023,
  0.05800000000000001,
  0.096,
  0.136,
  0.18899999999999997,
  0.256,
  0.398,
  0.821,
  0.9739999999999999])

In [116]:
auc, tpr, fpr = return_auc_attr(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.20858249999999998,
 [0.145, 0.145, 0.146, 0.146, 0.147, 0.15, 0.163, 0.271, 0.541],
 [0.019,
  0.057,
  0.105,
  0.146,
  0.212,
  0.277,
  0.429,
  0.8219999999999998,
  0.9739999999999999])

In [117]:
auc, tpr, fpr = return_auc_attr(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.1296965,
 [0.051,
  0.052000000000000005,
  0.052000000000000005,
  0.052000000000000005,
  0.054000000000000006,
  0.05600000000000001,
  0.061,
  0.191,
  0.561],
 [0.02,
  0.054000000000000006,
  0.08900000000000001,
  0.127,
  0.196,
  0.266,
  0.391,
  0.826,
  0.971])

In [118]:
auc, tpr, fpr = return_auc_attr(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.1946695,
 [0.001, 0.001, 0.002, 0.004, 0.01, 0.017, 0.053, 0.413, 0.894],
 [0.028000000000000004,
  0.05800000000000001,
  0.092,
  0.151,
  0.22400000000000003,
  0.276,
  0.435,
  0.828,
  0.975])

# BIM

In [119]:
adv_path1 = 'adv samples/IMAGENET/ResNet50/BIM/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/IMAGENET/ResNet50/BIM/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/IMAGENET/ResNet50/BIM/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/IMAGENET/ResNet50/BIM/0.25098039215686274eps.npz'

In [120]:
auc, tpr, fpr = return_auc_logit(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.2402875,
 [0.141,
  0.17299999999999996,
  0.199,
  0.206,
  0.241,
  0.291,
  0.3509999999999999,
  0.486],
 [0.016, 0.088, 0.138, 0.16, 0.276, 0.428, 0.588, 0.821])

In [121]:
auc, tpr, fpr = return_auc_logit(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.3262315,
 [0.342,
  0.355,
  0.36199999999999993,
  0.368,
  0.385,
  0.403,
  0.42100000000000004,
  0.498],
 [0.02, 0.095, 0.161, 0.182, 0.307, 0.449, 0.587, 0.824])

In [122]:
auc, tpr, fpr = return_auc_logit(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.45005399999999995,
 [0.532, 0.534, 0.539, 0.539, 0.546, 0.551, 0.558, 0.592],
 [0.017,
  0.078,
  0.15,
  0.17299999999999996,
  0.281,
  0.43799999999999994,
  0.585,
  0.829])

In [123]:
auc, tpr, fpr = return_auc_logit(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.4743655,
 [0.578, 0.579, 0.58, 0.58, 0.581, 0.587, 0.591, 0.622],
 [0.02, 0.09300000000000001, 0.152, 0.171, 0.286, 0.434, 0.594, 0.824])

In [124]:
auc, tpr, fpr = return_auc_attr(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.2642709999999999,
 [0.034,
  0.037,
  0.046,
  0.054000000000000006,
  0.065,
  0.078,
  0.13,
  0.4640000000000001,
  0.806],
 [0.024,
  0.05,
  0.086,
  0.133,
  0.19,
  0.237,
  0.37200000000000005,
  0.8080000000000002,
  0.98])

In [125]:
auc, tpr, fpr = return_auc_attr(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.189743,
 [0.063,
  0.064,
  0.067,
  0.072,
  0.078,
  0.09300000000000001,
  0.11899999999999998,
  0.313,
  0.684],
 [0.02, 0.055, 0.1, 0.142, 0.209, 0.267, 0.44, 0.836, 0.972])

In [126]:
auc, tpr, fpr = return_auc_attr(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.1734345,
 [0.079,
  0.081,
  0.08200000000000002,
  0.084,
  0.085,
  0.08900000000000001,
  0.09700000000000002,
  0.263,
  0.589],
 [0.025, 0.051, 0.084, 0.125, 0.18099999999999997, 0.243, 0.393, 0.82, 0.971])

In [127]:
auc, tpr, fpr = return_auc_attr(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.16003949999999997,
 [0.073, 0.073, 0.073, 0.074, 0.074, 0.074, 0.083, 0.235, 0.611],
 [0.024,
  0.055,
  0.08,
  0.125,
  0.16699999999999998,
  0.23200000000000004,
  0.37,
  0.829,
  0.973])

# CW

In [128]:
adv_path1 = 'adv samples/IMAGENET/ResNet50/CW/0.15eps.npz'

In [129]:
auc, tpr, fpr = return_auc_logit(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.7738314833501514,
 [0.8708375378405652,
  0.9434914228052471,
  0.9667003027245207,
  0.9677093844601413,
  0.9808274470232089,
  0.9868819374369324,
  0.9899091826437941,
  0.9929364278506561],
 [0.022, 0.092, 0.157, 0.18099999999999997, 0.309, 0.43, 0.567, 0.815])

In [130]:
auc, tpr, fpr = return_auc_attr(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.3628027245206862,
 [0.19273461150353183,
  0.20484359233097885,
  0.2119071644803229,
  0.22401614530776992,
  0.24318869828456105,
  0.26437941473259335,
  0.3158425832492432,
  0.5327951564076691,
  0.6468213925327951],
 [0.027000000000000003,
  0.066,
  0.10800000000000001,
  0.148,
  0.206,
  0.261,
  0.3990000000000001,
  0.813,
  0.973])

# ImageNet: MobileNet

In [131]:
# load mobilet model first

def load_imagenet_model():
    model=torchvision.models.mobilenet_v3_small(weights=True).to(device)
    model.to('cuda')
    model.train(False)
    return model

In [132]:
normal_model = load_imagenet_model()
normal_model.to(device)
normal_model.eval()

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [133]:
def return_auc_logit(adv_path, model):
    
    
    k=[1810,2510,2810,3000, 3200,3500, 6000]
    l=[6700,6700,6700,6700,6700,6700, 6700]
    
    compute_metrics_benign(adv_path, model)
    compute_metrics_adv(adv_path, model)
    df_cifar = pd.read_csv("adaptive_Benign.csv")
    attr_gaussian3 = df_cifar.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3 = df_cifar.iloc[6].values.flatten().tolist()[1:]
        
    fpr_results =[]
    for t1,t2 in zip(k,l):
        FPR = compute_FPR(logit_gaussian3, t1,t2)
        fpr_results.append(FPR/100)
        
    df_pgd_eps1 = pd.read_csv("adaptive_Adv.csv")
    attr_gaussian3_eps1 = df_pgd_eps1.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3_eps1 = df_pgd_eps1.iloc[6].values.flatten().tolist()[1:]
    
    tpr_results =[]
    for t1,t2 in zip(k,l):
        TPR = compute_TPR(logit_gaussian3_eps1, t1,t2)
        tpr_results.append(TPR/100)
    return(sklearn.metrics.auc(fpr_results, tpr_results), tpr_results, fpr_results)

In [134]:
def return_auc_attr(adv_path, model):
    
    #attrgaussian3
    m=[1400,1750,1900, 2300, 3200, 3500, 6000]
    n=[6600,6600,6600,6600,6600,6600,6600]

    
    compute_metrics_benign(adv_path, model)
    compute_metrics_adv(adv_path, model)
    df_cifar = pd.read_csv("adaptive_Benign.csv")
    attr_gaussian3 = df_cifar.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3 = df_cifar.iloc[6].values.flatten().tolist()[1:]
        
    fpr_results =[]
    for t3,t4 in zip(m,n):
        FPR = compute_FPR(attr_gaussian3,t3,t4)
        fpr_results.append(FPR/100)
        
    df_pgd_eps1 = pd.read_csv("adaptive_Adv.csv")
    attr_gaussian3_eps1 = df_pgd_eps1.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3_eps1 = df_pgd_eps1.iloc[6].values.flatten().tolist()[1:]
    
    tpr_results =[]
    for t3,t4 in zip(m,n):
        TPR = compute_TPR( attr_gaussian3_eps1,t3,t4)
        tpr_results.append(TPR/100)
    return(sklearn.metrics.auc(fpr_results, tpr_results), tpr_results, fpr_results)

# FGSM

In [135]:

adv_path1 = 'adv samples/IMAGENET/MobileNet/FGSM/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/IMAGENET/MobileNet/FGSM/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/IMAGENET/MobileNet/FGSM/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/IMAGENET/MobileNet/FGSM/0.25098039215686274eps.npz'

In [136]:
auc, tpr, fpr = return_auc_logit(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.7095309999999999,
 [0.051, 0.292, 0.42100000000000004, 0.506, 0.584, 0.698, 0.998],
 [0.02, 0.05600000000000001, 0.095, 0.132, 0.195, 0.292, 0.971])

In [137]:
auc, tpr, fpr = return_auc_logit(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.780073,
 [0.069, 0.405, 0.563, 0.643, 0.731, 0.8269999999999998, 0.9990000000000001],
 [0.015, 0.068, 0.114, 0.165, 0.21600000000000003, 0.31, 0.9739999999999999])

In [138]:
auc, tpr, fpr = return_auc_logit(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.8648105000000001,
 [0.177, 0.643, 0.7559999999999999, 0.831, 0.883, 0.934, 0.9990000000000001],
 [0.014000000000000002,
  0.061,
  0.11200000000000002,
  0.147,
  0.21899999999999997,
  0.313,
  0.97])

In [139]:
auc, tpr, fpr = return_auc_logit(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9302929999999999,
 [0.4109999999999999, 0.86, 0.935, 0.9620000000000001, 0.98, 0.992, 1.0],
 [0.008, 0.043, 0.102, 0.144, 0.199, 0.307, 0.965])

In [140]:
auc, tpr, fpr = return_auc_attr(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.8033315,
 [0.235, 0.607, 0.731, 0.916, 0.997, 0.9990000000000001, 1.0],
 [0.034, 0.16400000000000003, 0.245, 0.477, 0.856, 0.911, 0.997])

In [141]:
auc, tpr, fpr = return_auc_attr(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.8946705000000001,
 [0.536, 0.88, 0.935, 0.988, 1.0, 1.0, 1.0],
 [0.051, 0.166, 0.251, 0.501, 0.878, 0.924, 0.9990000000000001])

In [142]:
auc, tpr, fpr = return_auc_attr(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9488185,
 [0.9, 0.993, 0.998, 1.0, 1.0, 1.0, 1.0],
 [0.044, 0.166, 0.255, 0.509, 0.862, 0.9179999999999999, 1.0])

In [143]:
auc, tpr, fpr = return_auc_attr(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9487275,
 [0.995, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.049, 0.158, 0.221, 0.462, 0.8439999999999999, 0.8960000000000001, 0.998])

# PGD

In [144]:
adv_path1 = 'adv samples/IMAGENET/MobileNet/PGD/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/IMAGENET/MobileNet/PGD/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/IMAGENET/MobileNet/PGD/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/IMAGENET/MobileNet/PGD/0.25098039215686274eps.npz'

In [145]:
auc, tpr, fpr = return_auc_logit(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.947166,
 [0.986, 0.986, 0.986, 0.986, 0.986, 0.986, 0.997],
 [0.015, 0.06, 0.11, 0.156, 0.20800000000000002, 0.324, 0.972])

In [146]:
auc, tpr, fpr = return_auc_logit(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.954,
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.017, 0.06, 0.10800000000000001, 0.155, 0.215, 0.32300000000000006, 0.971])

In [147]:
auc, tpr, fpr = return_auc_logit(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9600000000000001,
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.016, 0.06, 0.117, 0.16, 0.215, 0.307, 0.976])

In [148]:
auc, tpr, fpr = return_auc_logit(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.951,
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.018, 0.063, 0.115, 0.154, 0.214, 0.313, 0.9689999999999999])

In [149]:
auc, tpr, fpr = return_auc_attr(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.34867100000000006,
 [0.039, 0.092, 0.133, 0.287, 0.654, 0.747, 0.992],
 [0.062, 0.187, 0.269, 0.516, 0.848, 0.8960000000000001, 0.9990000000000001])

In [150]:
auc, tpr, fpr = return_auc_attr(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.33144300000000004,
 [0.017, 0.07, 0.096, 0.234, 0.643, 0.752, 0.993],
 [0.057, 0.192, 0.262, 0.477, 0.85, 0.9079999999999999, 1.0])

In [151]:
auc, tpr, fpr = return_auc_attr(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.405738,
 [0.03, 0.083, 0.16, 0.364, 0.787, 0.8640000000000001, 0.996],
 [0.05, 0.18099999999999997, 0.276, 0.504, 0.867, 0.92, 1.0])

In [152]:
auc, tpr, fpr = return_auc_attr(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.622031,
 [0.11600000000000002, 0.365, 0.475, 0.704, 0.907, 0.9280000000000002, 0.998],
 [0.055, 0.17800000000000002, 0.261, 0.523, 0.855, 0.92, 0.9990000000000001])

# BIM

In [153]:
adv_path1 = 'adv samples/IMAGENET/MobileNet/BIM/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/IMAGENET/MobileNet/BIM/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/IMAGENET/MobileNet/BIM/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/IMAGENET/MobileNet/BIM/0.25098039215686274eps.npz'

In [154]:
auc, tpr, fpr = return_auc_logit(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.5397735,
 [0.269,
  0.31,
  0.3390000000000001,
  0.36199999999999993,
  0.38,
  0.42199999999999993,
  0.865],
 [0.016, 0.063, 0.113, 0.16, 0.206, 0.298, 0.981])

In [155]:
auc, tpr, fpr = return_auc_logit(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.6466895,
 [0.512, 0.529, 0.539, 0.548, 0.558, 0.575, 0.894],
 [0.016, 0.071, 0.125, 0.166, 0.226, 0.306, 0.971])

In [156]:
auc, tpr, fpr = return_auc_logit(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.7575684999999999,
 [0.726, 0.728, 0.731, 0.733, 0.737, 0.741, 0.894],
 [0.017, 0.062, 0.125, 0.17, 0.212, 0.312, 0.9739999999999999])

In [157]:
auc, tpr, fpr = return_auc_logit(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.8182235,
 [0.8080000000000002, 0.811, 0.812, 0.812, 0.815, 0.8169999999999998, 0.92],
 [0.014000000000000002, 0.057, 0.12, 0.163, 0.215, 0.31, 0.975])

In [158]:
auc, tpr, fpr = return_auc_attr(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.25904499999999997,
 [0.013000000000000001, 0.025, 0.038, 0.129, 0.538, 0.6579999999999999, 0.991],
 [0.034, 0.172, 0.241, 0.463, 0.857, 0.917, 1.0])

In [159]:
auc, tpr, fpr = return_auc_attr(adv_path2, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.232611,
 [0.016, 0.033, 0.044, 0.127, 0.461, 0.576, 0.992],
 [0.052000000000000005,
  0.176,
  0.253,
  0.48100000000000004,
  0.86,
  0.9129999999999999,
  1.0])

In [160]:
auc, tpr, fpr = return_auc_attr(adv_path3, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.2096805,
 [0.011, 0.019, 0.024, 0.09300000000000001, 0.425, 0.561, 0.99],
 [0.057, 0.169, 0.241, 0.4640000000000001, 0.857, 0.917, 0.997])

In [161]:
auc, tpr, fpr = return_auc_attr(adv_path4, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.281789,
 [0.008, 0.027000000000000003, 0.048, 0.165, 0.586, 0.699, 0.995],
 [0.046, 0.175, 0.252, 0.489, 0.846, 0.91, 1.0])

# CW

In [163]:
adv_path1 = 'adv samples/IMAGENET/MobileNet/CW/0.15eps.npz'

auc, tpr, fpr = return_auc_logit(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9195945,
 [0.583,
  0.8539999999999999,
  0.9230000000000002,
  0.948,
  0.9670000000000001,
  0.98,
  0.998],
 [0.02, 0.063, 0.117, 0.171, 0.231, 0.301, 0.973])

In [164]:
adv_path1 = 'adv samples/IMAGENET/MobileNet/CW/0.15eps.npz'

auc, tpr, fpr = return_auc_attr(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.4230565,
 [0.252, 0.282, 0.296, 0.36700000000000005, 0.627, 0.716, 0.976],
 [0.053, 0.18600000000000003, 0.259, 0.49, 0.86, 0.912, 0.996])